# Reconstruction Engines
**Learning more about the different engines and their parameters.**

PtyPy offers a range of different reconstruction engines that can be grouped into the 3 main categories of

- Projectional engines (DM, RAAR)
- Stochastic engines (ePIE, SDR)
- Gradient-based engines (ML)

These engine groups share similar sets of parameters. In this example, we pick one from each category and look at some of their parameters.

## Difference Map (DM) algorithm
In PtyPy we have implemented a generalised version of the difference map algorithm with the following update for the exit wave $\psi$ of every view

$$
\psi_{j+1} = \{\alpha (\mathbb{I} - \mathcal{O}) + \mathcal{F} [(1 + \alpha) \mathcal{O} - \alpha \mathbb{I}]\}(\psi_{j})
$$

where $\mathcal{O}$ and $\mathcal{F}$ are the overlap and Fourier update, respectively. $\mathbb{I}$ is the idenity matrix and $\alpha$ is a tuning parameter that blends pure alternating projections ($\alpha=0$) with the original DM algorithm ($\alpha=1$). By its very nature, $\alpha=1$ is more exploratory but can be a bit too "aggressive" in some cases, whereas $\alpha=0$ has a tendency to get stuck in local minima. In practice, $\alpha=0.95$ seems to be a good and robust compromise. In our moonflower example, we can adjust the `alpha` parameter accordingly

```python
p.engines.engine00 = u.Param()
p.engines.engine00.name = "DM"
p.engines.engine00.numiter = 80
p.engines.engine00.alpha = 0.95

```
### The Fourier update
During the Fourier update $\mathcal{F}[\Psi]$, the model $\Psi$ - transformed into the Fourier domain - is being updated such that its intensity is replaced by the measured data $I$ while its phase remains unchanged

$$
\mathcal{F}[\Psi] = \Psi \cdot \frac{\sqrt{|I|} + \rho \cdot (|\Psi| - \sqrt{|I|})}{|\Psi|}
$$

where $\rho$ is a renormalisation factor defined as

$$
\rho = \begin{cases}
         1  & \text{if } b \geq \langle(|\Psi| - \sqrt{|I|})^2\rangle \\
         \sqrt{b / \langle(|\Psi| - \sqrt{|I|})^2\rangle}  & \text{if } b < \langle(|\Psi| - \sqrt{|I|})^2\rangle \\
         0  & \text{if } b = \text{None}
         \end{cases}
$$

with $b$ being the so called power bound, which can be modified using the `fourier_power_bound` engine parameter. For Poisson-sampled data, the theoretical value for this power bound parameter is $0.25$.

```python
p.engines.engine00.fourier_power_bound = 0.25
```

### The overlap update
During the Overlap update $\mathcal{O}[\psi]$, the exit wave model $\psi$ - back-transformed into the real domain - is being used to update object $O$ and probe $P$

$$
O = \frac{\sum_k P^* \cdot \psi_k}{\sum_k|P|^2}
$$
$$
P = \frac{\sum_k O_k^* \cdot \psi_k}{\sum_k|O_k|^2}
$$
where $\sum_k$ is the summation over all views and $^*$ denotes the complex conjugate. Based on these updates, the new model is being updated such that

$$
\mathcal{O}[\psi]_k = P \cdot O_k
$$

for all views $k$. Since object and probe update depend on each other, both are repeated several times to reach convergence. In PtyPy, termination of this inner loop is triggered by reaching `overlap_max_iterations` or if the root mean square difference between the current and previous estimate of the probe is smaller than `overlap_converge_factor`. Additionally, the behaviour of the overlap update can be controlled by delaying the first update of the probe using `probe_update_first` or by swapping the order of the object and probe update using `update_object_first` which is `True` by default.

```python
p.engines.engine00.overlap_converge_factor = 0.05
p.engines.engine00.overlap_max_iterations = 10
p.engines.engine00.update_object_first = True
p.engines.engine00.probe_update_start = 2
```

`````{admonition} Exercise 
:class: attention
Modify different DM engine parameters and observe their impact on the outcome of the reconstruction.
`````

---

In [ ]:
import ptypy
import ptypy.utils as u

p = u.Param()
p.verbose_level = "interactive"
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Live-plotting
p.io.autoplot = u.Param()
p.io.autoplot.active=True
p.io.autoplot.threaded = False
p.io.autoplot.layout = "jupyter"
p.io.autoplot.interval = 10

p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = "Full"
p.scans.MF.data= u.Param()
p.scans.MF.data.name = "MoonFlowerScan"
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.

# Define reconstruction engines
p.engines = u.Param()

# Difference Map (DM) engine
p.engines.engine00 = u.Param()
p.engines.engine00.name = "DM"
p.engines.engine00.numiter = 80

# General update
p.engines.engine00.alpha = 0.95

# Fourier update
p.engines.engine00.fourier_power_bound = 0.25

# Overlap update
p.engines.engine00.overlap_converge_factor = 0.05
p.engines.engine00.overlap_max_iterations = 10
p.engines.engine00.update_object_first = True
p.engines.engine00.probe_update_start = 2

P = ptypy.core.Ptycho(p,level=5)

## ePIE algorithm

The exit wave update in standard ePIE is basically like alternating projections in that

$$
\Psi_{j+1} = \mathcal{F} (\mathcal{O}(\Psi_{j}))
$$

where $\mathcal{O}$ and $\mathcal{F}$ are again the overlap and Fourier update, respectively. In contrast to projectional methods ePIE is following a stochastic approach where the exit wave is updated sequentially for randomly selected views. In each of these randomized steps as part of the overlap update, the probe $P_{j}$ and object $O_{j}$ are modified like this 

$$
O_{j+1} = O_{j} + \alpha \cdot P_{j}^{*} \cdot [\Psi_{\prime} - \Psi_{j+1}] / ||P_{j}||_{max}^2
$$ 

$$
P_{j+1} = P_{j} + \beta \cdot O_{j+1}^{*} \cdot [\Psi_{\prime} - \Psi_{j+1}] / ||O_{j}||_{max}^2
$$ 

where $\Psi_{\prime} = \mathcal{O}(\Psi_{j}) = P_{j} \cdot O_{j}$ is the exit wave from the previously updated view, $^{*}$ is the complex conjugate and $||\cdot||_{max}$ is the maximum norm. The parameters $\alpha$ and $\beta$ control the strength of object and probe update, respectively and are both equal to $1$ by default. Lowering $\alpha$ or $\beta$ will lower the impact of the object/probe update in each stochastic update, which slows down convergence but might help in escaping artefacts that can occur in some cases. In practice, $\alpha=0.1$ and $\beta=0.9$ seems to be a good alternative choicefor for some tricky experimental datasets. In our moonflower example we can stick to the default values for `alpha` and `beta`

```python
p.engines.engine00 = u.Param()
p.engines.engine00.name = "EPIE"
p.engines.engine00.numiter = 200
p.engines.engine00.alpha = 1
p.engines.engine00.beta = 1
```

During the probe update of the original ePIE algorithm, the maximum norm is calculated and applied seperately for each view which can lead to big changes in the probe depending on the random sequence of views. A different approach is to always calculate the maximum norm over the entire object resulting in less jumpy changes during the probe update. This can be controlled using `object_norm_is_global` which is `False` by default.

```python
p.engines.engine00.object_norm_is_global = False

```

`````{admonition} Exercise 
:class: attention
Modify different ePIE engine parameters and observe their impact on the outcome of the reconstruction.
`````

---

In [ ]:
import ptypy
import ptypy.utils as u

p = u.Param()
p.verbose_level = "interactive"
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Live-plotting
p.io.autoplot = u.Param()
p.io.autoplot.active=True
p.io.autoplot.threaded = False
p.io.autoplot.layout = "jupyter"
p.io.autoplot.interval = 10

p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = "Full"
p.scans.MF.data= u.Param()
p.scans.MF.data.name = "MoonFlowerScan"
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.

# Define reconstruction engines
p.engines = u.Param()

# ePIE engine
p.engines.engine00 = u.Param()
p.engines.engine00.name = "EPIE"
p.engines.engine00.numiter = 200
p.engines.engine00.numiter_contiguous = 10
p.engines.engine00.alpha = 1
p.engines.engine00.beta = 1
p.engines.engine00.object_norm_is_global = False

P = ptypy.core.Ptycho(p,level=5)

## Maximum Likelihood (ML) algorithm

In contrast to the projectional and stochastic algorithms, the maximum-likelihood approach allows to explicitely define a noise model for the ptychographic reconstruction. 
For ptychography, the negative log-likelihood can be written as

$$
\mathcal{L} = - \log \prod_k \prod_q p(I_{kq}|PO) 
$$

where $p(I_{kq}|PO)$ is the probability of measuring a photon $I_{kq}$ at scan view $k$ and detector pixel $q$ given probe $P$ and object $O$. For a Gaussian distribution the likelihood reduces to a sum of least squares

$$
\mathcal{L} = \sum_k \sum_q \frac{(|\Psi_{kq}|^2 - I_{kq})^2}{2\sigma^2_{kq}}
$$

which can be used to minimise with respect to $P$ and $O$ using a non-linear conjugate gradient (CG) approach by defining the gradient

$$
g = \left(\frac{\partial \mathcal{L}}{\partial P}, \frac{\partial\mathcal{L}}{\partial O}\right)
$$

which in turn can be used to define the new conjugate search direction for the $n-th$ iteration step

$$
\Delta^{(n)} = -g^{(n)} + \beta^{(n)}\Delta^{(n-1)}
$$
 
where $\beta^{(n)}$ can be calculated via the Polak-Ribiere formula. For more details about this maximum-likeilihood based CG algorithm, see [Thibault P. and Guizar-Sicairos M. (2012)](http://dx.doi.org/10.1088/1367-2630/14/6/063004) which also includes a description of preconditioners and regularisers that can help with the convergence of the algorithm. In particular, the scale preconditioner can be useful for weakly scattering objects - assuming that the object deviates only weakly from unity - by enforcing a near equality between the probe and object gradients. This parameter can be controlled using `scale_precond` which is `True` by default and `scale_probe_object` which is $1.0$ by default.
```python
p.engines.engine00.scale_precond = True
p.engines.engine00.scale_probe_object = 1.
```
In addition, regularisers can be used to reduce difficulties that can arise from ill-posed problems such as the inverse phase problem in ptychography. To impose some form of smoothness in the final reconstruction, we can use a simple Gaussian regulariser which can be toggled using `reg_del2` (`True` by default) and tuned using `reg_del2_amplitude` which defaults to $0.01$. In this moonflower example, we are changing the amplitude to $1.$ for a more aggressive smoothing strategy. 

```python
p.engines.engine00.reg_del2 = True 
p.engines.engine00.reg_del2_amplitude = 1.
```

`````{admonition} Exercise 
:class: attention
Modify different ML engine parameters and observe their impact on the outcome of the reconstruction.
`````

---

In [ ]:
import ptypy
import ptypy.utils as u

p = u.Param()
p.verbose_level = "interactive"
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Live-plotting
p.io.autoplot = u.Param()
p.io.autoplot.active=True
p.io.autoplot.threaded = False
p.io.autoplot.layout = "jupyter"
p.io.autoplot.interval = 10

p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = "Full"
p.scans.MF.data= u.Param()
p.scans.MF.data.name = "MoonFlowerScan"
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.

# Define reconstruction engines
p.engines = u.Param()

# Maximum Likelihood (ML) engine
p.engines.engine00 = u.Param()
p.engines.engine00.name = "ML"
p.engines.engine00.ML_type = "Gaussian"
p.engines.engine00.numiter = 300
p.engines.engine00.numiter_contiguous = 10
p.engines.engine00.reg_del2 = True 
p.engines.engine00.reg_del2_amplitude = 1.
p.engines.engine00.scale_precond = True
p.engines.engine00.scale_probe_object = 1.

P = ptypy.core.Ptycho(p,level=5)

## GPU-accelerated engines

For running the GPU-accelerated equivalent of the engines described above, simple add this line at the top of the script

```python
ptypy.load_gpu_engines("cuda")
```

and add "_pycuda" to the engine name, for example

```python
# Difference Map (DM) engine
p.engines.engine00 = u.Param()
p.engines.engine00.name = "DM_pycuda"
```

`````{admonition} Exercise 
:class: attention
Run the GPU-accelerated version of the above examples with different engines and observe the increase in reconstruction speed.
`````

---

In [ ]:
import ptypy
import ptypy.utils as u

ptypy.load_gpu_engines("cuda")

# Copy the parameter tree here
# ...